In [ ]:
op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
op.columns = [re.sub('\w+\.', '', i) for i in list(op.columns)]

In [ ]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
df = pd.read_csv('./dws_dsc_customer_opportunity_monthly_kpi_1.csv')
op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
op.columns = [re.sub('\w+\.', '', i) for i in list(op.columns)]
df = df.fillna(0)
# df.columns
df['if_cooped'] = 0
df['if_cooped'] = df['if_cooped'].where(df['contract_signed_opportunity_num'] < 1, 1) # 今年有没合作。
cus_bf_21 = op[(op['createdon'] < '2021-11-01') & (op['milestone'] == 'Contract Signed')]['account_name_en'].unique()
cus_bf_21 = list(cus_bf_21)
cus_af_21 = list(df['customer_name'].unique())
df['old_cust'] = df['customer_name'].isin(cus_bf_21)
# df.to_csv('./data_down/dws_dsc_customer_opportunity_monthly_kpi_1.csv', index = None, encoding='utf_8_sig')

df['annual_average_revenue'] = df['annual_average_revenue'] * 1000
df['annual_average_gross_profit'] = df['annual_average_gross_profit'] * 1000

op = op[op['annual_average_gross_profit'].str.match('(\d+\.)').fillna(False)]
op = op[op['annual_average_revenue'].str.match('(\d+\.)').fillna(False)]

op['annual_average_revenue'] = op['annual_average_revenue'].astype(float) * 1000
op['annual_average_gross_profit'] = op['annual_average_gross_profit'].astype(float) * 1000
colors = sns.color_palette('rocket')[0:5]
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plt1
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']

In [ ]:
def plot_pie(plt10, plt11,  bi_title, **kwargs):
    """
    成功, 总和, 
    """
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(
        go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
        labels = ['succeed', 'fail']),
        1, 1)

    fig.add_trace(
        go.Pie(values = [plt10, plt11 - plt10], 
        labels = ['succeed', 'fail']),
        1, 2)
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_layout(title_text=bi_title,
    width=900,
    height=400,
    margin=dict(l=10, r=10, t=50, b=20),
    paper_bgcolor="LightSteelBlue",)

    return fig
 

In [ ]:
plt10 = sum(df['contract_signed_opportunity_num'])  # win
plt11 = sum(df['total_opportunity_num'] - df['new_business_pipeline_unclosed_opp_num'])

# sum(df['total_opportunity_num'] - df['closed_canceled_opportunity_num'] - df['closed_lost_opportunity_num']) # 整体的商机成功率.a
# plot_pie(plt10, plt11,  "整体的商机本年成功率\t\n 成功数量")

In [ ]:
# ** 从年初到现在  总体 annual_average_revenue**
plt12 = df.query("total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum() # win
plt13 = df['annual_average_revenue'].sum() 
# plot_pie(plt12, plt13,  "整体的商机本年成功金额占比 \t\n 成功金额")

In [ ]:
df.shape[0] # 今年所有客户数量

In [ ]:
plt20 = df.query("old_cust == 1 & if_cooped == 1 & total_opportunity_num > 0").shape[0] 
plt21 = df.query("old_cust == 1 & total_opportunity_num > 0").shape[0] 
# 0.592375366568915 旧客户留存率。
# plot_pie(plt20, plt21, '老客户新生意签约情况 \n\t 老客户新生意签约数量')
# christine
# 老客户新生意 *老客户老生意。*

In [ ]:
# opportunity win & contract signed 区别.
plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['contract_signed_opportunity_num'].sum()
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")
plt31 = sum(plt31['total_opportunity_num'] - plt31['new_business_pipeline_unclosed_opp_num'])
# sum(plt31['total_opportunity_num'] - plt31['closed_canceled_opportunity_num'] - plt31['closed_lost_opportunity_num'])

# plot_pie(plt30, plt31,  "商机level: 已经合作过的客户本年商机成功率\t\n 成功数量")

In [ ]:
# ** 从年初到现在老的客户为我们带来的新的商机的 annual_average_revenue 收益 **
plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum() 
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['annual_average_revenue'].sum()
# plot_pie(plt30, plt31,  "商机level: 已经合作过的客户本年成功金额占比 \t\n 成功金额")

In [ ]:
# df.query("old_cust == 0 & total_opportunity_num > 0 & ")['total_opportunity_num'].sum() 

In [ ]:
# [df.query("old_cust == 1")['total_opportunity_num'].sum(), df.query("old_cust == 1")['total_opportunity_num'].sum() / sum(df['total_opportunity_num'])]
plt32 = df.query("old_cust == 0 & total_opportunity_num > 0")['contract_signed_opportunity_num'].sum() 
plt33 = df.query("old_cust == 0 & total_opportunity_num > 0") # xinke zong 
plt33 =   sum(plt33['total_opportunity_num'] - plt33['new_business_pipeline_unclosed_opp_num'])

# sum(plt33['total_opportunity_num'] - plt33['closed_canceled_opportunity_num'] - plt33['closed_lost_opportunity_num'])
  
# plot_pie(plt32, plt33, '商机level: 新客户本年商机成功率 \n\t 新客户本年商机成功数')
    # open 的要去掉

In [ ]:
# ** 从年初到现在新增的客户为我们带来的 annual_average_revenue 收益 **
plt36 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum()
plt37 = df.query("old_cust == 0 & total_opportunity_num > 0")['annual_average_revenue'].sum() * .18# xinke zong 
plot_pie(plt36, plt37, '商机level: 新客户本年成功金额占比 \t\n 成功金额')

In [ ]:
op['opportunity_record'].value_counts()
# 1089 na

In [255]:
df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['contract_signed_opportunity_num'].sum() 

37

In [ ]:
# # open 商机下, 前三十客户以及对应商机金额. 目前最应该重视的客户& 商机
# plt38 = op[op['milestone'].isin(mst_list) ]
# plt38['annual_average_gross_profit'] = plt38['annual_average_gross_profit'].astype(float)  
# plt38 = plt38.sort_values('annual_average_gross_profit', ascending=False).head(30)
# px.bar(plt38, x = 'account_name_en', y = 'annual_average_gross_profit')

In [ ]:
# op['opportunity_record']

In [ ]:
# renew 型的商机; 到renew 期限的时候, 适当提醒.
plt39 = op.fillna(0).query("opportunity_record != 0 & opportunity_record != 'BCA Not Required'")
# plt39 = op[op['milestone'].isin(mst_list) ]
# plt39 = plt39[plt39['annual_average_gross_profit'].str.contains('[azAZ]').fillna(True)] 
plt39['annual_average_gross_profit'] = plt39['annual_average_gross_profit'].astype(float)
plt39 = plt39.sort_values('annual_average_gross_profit', ascending=False).head(30)
px.bar(plt39, x = 'account_name_en', y = 'annual_average_gross_profit', color = 'opportunity_record')


---
# 客户

In [ ]:
# 此前合作过的客户

plt42 = len(set(np.intersect1d(cus_bf_21, cus_af_21)))
plt43 = len(cus_af_21)

plot_pie(plt42, plt43, '客户level :  本年有商机的account占比 \n\t 老account数')
# """# [341, 717]  今年出现的过往的客户 vs 今年的所有"""

In [ ]:
# 此前未合作过的客户
plt40 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['customer_name'].nunique()
plt41 = df.query("old_cust == 0 & total_opportunity_num > 0")['customer_name'].nunique()
plot_pie(plt40, plt41, 'account level :  account转化率 \n\t account转化数')

# account

In [ ]:
# a0 = df['annual_average_revenue'].sum();a0 
# a1 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();a1
# b = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();b
# (a1 + b )/ a0 # 百分之四十的金额 被得到.

--- 
## ci milestone 和对应的contract value 的相关性

In [ ]:
opp = op[['account_name_en','annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
opp = opp.fillna(0)
opp = opp[opp['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
opp = opp[opp['contract_term'] != 0] 
# opp = op
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']
sctr_list = ['Other', 0]
prodc_list = [0]

opp = opp[opp['milestone'].isin(mst_list)]
# opp = opp[~opp['sector'].isin(sctr_list)]
# opp = opp[~opp['product1'].isin(prodc_list)]

opp = opp[opp['annual_average_revenue'] != 0]
opp['annual_average_revenue_log'] = np.log(op['annual_average_revenue'])
px.scatter(opp, x = 'contract_term', y = 'annual_average_revenue_log',\
     color="milestone", trendline = 'ols')



` 1. milestone , 我们更容易签署周期更短的合同` <br>
` 2. 短期合同中, 我们赢得的合同比输掉的合同普遍价值更高. ` # ninahua

In [ ]:
# import statsmodels.formula.api as smf
# smf.ols('annual_average_revenue ~  milestone -1 ', data = op).fit().summary()


`商机金额大小与转化率的关系。 c 不显著#`

In [ ]:
# 商机金额大小与转化率的关系。 c
# df = df[(df['milestone'] != 'Early Lead') & (df['milestone'] != 'Potential Opportunity')]
def load_opp(df):
     df['win_or_loss'] = 0
     df['win_or_loss'] = df['win_or_loss'].where(df['milestone']  != 'Contract Signed', 1)
     df = df.fillna(0)
     # 删除有汉字的。
     df = df[~df['annual_average_gross_profit'].str.contains('[azAZ]').fillna(False)]
     df = df[~df['annual_average_revenue'].str.contains('[azAZ]').fillna(False)]
     return df
opp = load_opp(opp)
smf.logit(formula = "win_or_loss ~ contract_value ", data = opp).fit().summary()


In [ ]:
opp['milestone'].unique()

In [ ]:
# # violin
# px.violin(x = opp['win_or_loss'], y =  np.log(opp['contract_value']), \
#     points="all", color = opp['win_or_loss'], box = True )

### open shangji 
> open 商机可能带来 8743745010.0 的收入.

In [ ]:
# go.Figure(go.Sunburst(
#     labels=[ "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
#     parents=[  "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
#     values=[  14, 12, 10, 2, 6, 6, 4, 4],
# ))
opp = op[['account_name_en','annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
opp = opp.fillna(0)
opp = opp[opp['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
opp = opp[opp['contract_term'] != 0] 
opp[~opp['milestone'].isin(mst_list)]['annual_average_revenue'].astype(float).sum()
 

---
# sap

In [ ]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
sap = pd.read_csv('./dws_dsc_customer_operation_monthly_kpi.csv', sep = '\001')
# op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
sap.columns = [re.sub('\w+\.', '', i) for i in list(sap.columns)] 
sap = sap.fillna(0)
sap['ytm_cogs'] = sap['ytm_income_amt'] - sap['ytm_gross_profit_amt']
sap['ytm_gp_margin'] = sap['ytm_gross_profit_amt'] / sap['ytm_income_amt'] 
sap['ytm_gp_margin'].replace([-np.inf, np.inf],  0, inplace = True)
sap['sap_lb_cost'] = sap[[
    'ytm_perm_hr_cost','ytm_service_outsourcing_amt','ytm_labor_service_outsourcing_amt',]].sum(axis = 1)
sap['customer_type'] = sap['customer_type'].replace('Q3', 'Q5').replace('Q4', 'Q3').replace('Q5', 'Q4')
sap= sap.query('ytm_gp_margin > -3 & ytm_gp_margin < 3 ') 
# df.columns 

# labor fee
# perm_hr_cost



In [ ]:
# # def plot_pie(plt10, plt11,  bi_title, **kwargs):
# #     """
# #     成功, 总和, 
# #     """
# #     colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
# #     fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
# #     fig.add_trace(
# #         go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
# #         labels = ['succeed', 'fail']),
# #         1, 1)

# #     fig.add_trace(
# #         go.Pie(values = [plt10, plt11 - plt10], 
# #         labels = ['succeed', 'fail']),
# #         1, 2)
# #     fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
# #                   marker=dict(colors=colors, line=dict(color='#000000', width=2)))
# #     fig.update_layout(title_text=bi_title,
# #     width=900,
# #     height=400,
# #     margin=dict(l=10, r=10, t=50, b=20),
# #     paper_bgcolor="LightSteelBlue",)

# #     return fig
# sap_ly = pd.concat([sap[['customer_name', 'bg_code', 'year', 'month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num']], 
#        sap[pd.Series(sap.columns)[pd.Series(sap.columns).str.contains('^ly')]]], axis = 1)
# sap_ly['year'] = 2020
# sap_ty = sap.iloc[:,0:28]
# sap_ly.columns = sap_ty.columns
# sap_long = pd.concat([sap_ty, sap_ly], axis = 0).drop(['month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num'], axis = 1)

In [ ]:
sap_plt = sap[~sap['customer_name'].str.contains('ISC').fillna(False)]
sap_plt = sap_plt[~sap_plt['customer_name'].str.contains('Overhead').fillna(False)]
sap_plt = sap_plt[sap_plt['customer_name'] != 0]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
scaled = scaler.fit_transform(np.array(sap_plt['ytm_income_amt']).reshape(-1,1))
sap_plt['ytm_income_amt_scaled'] = np.log(scaled)
sap_plt['abs_ytm_income_amt'] = np.abs(sap_plt['ytm_income_amt'])
sap_plt= sap_plt.query('ytm_gp_margin > -.5 & ytm_gp_margin < .5 ') 
sap_plt['sap_lb_cost'] = np.abs(sap_plt['sap_lb_cost'])
# sap_plt = sap_plt.query("income_amt_scaled > -4")

In [256]:
def bubble_plt(plot_data, x, color):
    if x == 'scaled':
        x = 'ytm_income_amt_scaled'
        vline = np.log(scaler.transform(np.array(10000000).reshape(-1,1)))[0][0]
    else:
        x = 'ytm_income_amt'
        vline = 10000000
    # y = y
    fig = px.scatter(
        data_frame= plot_data, x = x, y= 'ytm_gp_margin', hover_data=['customer_name'],
        size = 'sap_lb_cost', color = color, title='利润率y , 收入x, 人力成本 z, customer level')  
    
    fig.add_hline(y = .06, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
        annotation_text="2021 <br> GP_margin .06 <br><br />  " , 
        annotation_position="top right", annotation_opacity = .4, col = 1)
    fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
    return fig
bubble_plt(sap_plt, 'scaled',  'customer_type')

`人力成本高中低，对应的客户，其利润率有无显著差异？说明什么?` <br>
`高人力成本 ~ 高利润 & 利润率.`

In [ ]:
# bubble_plt(sap_plt, 'scaled', 'sap_lb_cost') 

In [257]:
# causal inference.
# `高人力成本 ~ 高利润率 .`?
smf.ols('sap_lb_cost ~ customer_type', data = sap).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            sap_lb_cost   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     5.319
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           0.000361
Time:                        13:38:51   Log-Likelihood:                -6384.0
No. Observations:                 355   AIC:                         1.278e+04
Df Residuals:                     350   BIC:                         1.280e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            2.387e+06   2.27e+06      1.051      0.294   -2.08e+06    6.85e+06
customer_type[T.Q1]    8.2e+06   3.25e+06      2.526      0.012    1.81e+06    1.46e+07
customer_type[T.Q2]  5.978e+06   3.23e+06      1.851      0.065   -3.72e+05    1.23e+07
customer_type[T.Q3] -1.826e+06   2.65e+06     -0.689      0.491   -7.04e+06    3.38e+06
customer_type[T.Q4] -1.784e+06   2.87e+06     -0.622      0.534   -7.42e+06    3.85e+06
==============================================================================
Omnibus:                      763.700   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1041829.237
Skew:                          15.361   Prob(JB):                         0.00
Kurtosis:                     266.609   Cond. No.                         7.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# smf.ols('ytm_gp_margin ~ sap_lb_cost-1', data = sap).fit().summary() #labor_fee

合并财务以及商务

In [ ]:
df_plt = df.merge(sap[['sap_lb_cost','labor_fee', 'customer_type', \
    'customer_name', 'ytm_gp_margin', 'ytm_gross_profit_amt']], on = 'customer_name')

In [ ]:
df_plt 

In [ ]:
df_plt['abs_sap_lb_cost'] = np.abs(df_plt['sap_lb_cost'])

In [259]:
# Q1到Q4不同类别的客户，商机成功率有无显著差异？
# 人力成本不同的客户，商机成功率有无显著差异？


# px.scatter(data_frame= df_plt, x = 'sap_lb_cost', y= 'opportunity_win_rate', hover_data=['customer_name'], 
#         size = 'abs_sap_lb_cost', color = 'customer_type', title='人力费用, 商机胜率, 客户类型')  



# px.violin(x = df_plt['customer_type'], y =  df_plt['sap_lb_cost'], \
#     points="all", color = df_plt['customer_type'], box = True )


    
# px.violin(x = df_plt['customer_type'], y =  df_plt['opportunity_win_rate'], \
#     points="all", color = df_plt['customer_type'], box = True )

In [301]:
# 客户类型, / 人力成本 商机胜率

smf.ols('opportunity_win_rate ~   customer_type - 1 ', data = df_plt).fit().summary()
# smf.ols('opportunity_win_rate ~  sap_lb_cost  - 1 ', data = df_plt).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     opportunity_win_rate   R-squared:                       0.038
Model:                              OLS   Adj. R-squared:                  0.008
Method:                   Least Squares   F-statistic:                     1.274
Date:                  Thu, 09 Dec 2021   Prob (F-statistic):              0.288
Time:                          14:11:39   Log-Likelihood:                -35.500
No. Observations:                   100   AIC:                             79.00
Df Residuals:                        96   BIC:                             89.42
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
customer_type[Q1]     0.2507      0.073      3.413      0.001       0.105       0.396
customer_type[Q2]     0.3708      0.094      3.939      0.000       0.184       0.558
customer_type[Q3]     0.4235      0.055      7.698      0.000       0.314       0.533
customer_type[Q4]     0.4071      0.075      5.421      0.000       0.258       0.556
==============================================================================
Omnibus:                       14.613   Durbin-Watson:                   2.139
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                7.069
Skew:                           0.447   Prob(JB):                       0.0292
Kurtosis:                       2.053   Cond. No.                         1.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# df_plt['sap_lb_cost'].describe()


In [298]:
# 人力成本占比越高的业务，越可能产生高的毛利? yes
smf.ols('ytm_gross_profit_amt ~ sap_lb_cost-1 ', data = df_plt).fit().summary() #sap_lb_cost

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     ytm_gross_profit_amt   R-squared (uncentered):                   0.569
Model:                              OLS   Adj. R-squared (uncentered):              0.565
Method:                   Least Squares   F-statistic:                              130.8
Date:                  Thu, 09 Dec 2021   Prob (F-statistic):                    8.35e-20
Time:                          14:10:10   Log-Likelihood:                         -1643.1
No. Observations:                   100   AIC:                                      3288.
Df Residuals:                        99   BIC:                                      3291.
Df Model:                             1                                                  
Covariance Type:              nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
sap_lb_cost     0.4911      0.043     11.436      0.000       0.406       0.576
==============================================================================
Omnibus:                       89.735   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1230.377
Skew:                           2.703   Prob(JB):                    6.71e-268
Kurtosis:                      19.312   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# p7

In [ ]:
# sap_plt['customer_type'].value_counts()

In [ ]:
sap['customer_name'].nunique()

In [ ]:
sap['customer_type'].value_counts()

In [ ]:
px.bar(data_frame= sap, x = 'customer_type',  hover_data=['customer_name'], 
    color = 'customer_type', title='customer type count')  

# Q4    133
# Q3     80
# 0      47
# Q1     46
# Q2     44

In [282]:
# sap_plt.columns

In [267]:
def bubble_plt(plot_data, x, color):
    if x == 'scaled':
        x = 'ytm_income_amt_scaled'
        vline = np.log(scaler.transform(np.array(10000000).reshape(-1,1)))[0][0]
    else:
        x = 'ytm_income_amt'
        vline = 10000000
    # y = y
    fig = px.scatter(
        data_frame= plot_data, x = x, y= 'ytm_gp_margin', hover_data=['customer_name'],
        size = 'abs_ytm_income_amt', color = color, title='利润率y , 收入x, 收入绝对值z customer level')  
    
    fig.add_hline(y = .06, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
        annotation_text="2021 <br> GP_margin .06 <br><br />  " , 
        annotation_position="top right", annotation_opacity = .4, col = 1)
    fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
    return fig
# bubble_plt(sap_plt, 'scaled',  'customer_type')

In [288]:
# p9
bubble_plt(sap_plt, 'scaled', 'is_operation') # has_vas 
# bubble_plt(sap_plt, 'scaled', 'has_vas') #  
# df_plt['has_vas']

In [293]:
# 不同仓储客户的分类
# 有无仓储业务，其利润率高低，有无显著差异？
# 不同仓储业务类型的客户，利润率有无显著差异？
# fee_type

smf.ols('ytm_gross_profit_amt ~ is_operation ', data = sap).fit().summary() #ytm_gross_profit_amt

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     ytm_gross_profit_amt   R-squared:                       0.009
Model:                              OLS   Adj. R-squared:                  0.006
Method:                   Least Squares   F-statistic:                     3.143
Date:                  Thu, 09 Dec 2021   Prob (F-statistic):             0.0771
Time:                          14:06:36   Log-Likelihood:                -5985.2
No. Observations:                   355   AIC:                         1.197e+04
Df Residuals:                       353   BIC:                         1.198e+04
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             1.644e+06   7.86e+05      2.091      0.037    9.79e+04    3.19e+06
is_operation[T.非操作型] -1.484e+06   8.37e+05     -1.773      0.077   -3.13e+06    1.62e+05
==============================================================================
Omnibus:                      266.762   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19044.428
Skew:                          -2.419   Prob(JB):                         0.00
Kurtosis:                      38.554   Cond. No.                         5.65
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [278]:
corr_vas = sap['has_vas'].replace('有VAS', 1).replace('无VAS', 0)
corr_vas = pd.concat([corr_vas, sap['ytm_gross_profit_amt']], axis = 1)
coach_corr = corr_vas.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [299]:
corr_lb = sap[['sap_lb_cost', 'ytm_gross_profit_amt']]

In [300]:
coach_corr = corr_lb.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

,sap_lb_cost,ytm_gross_profit_amt
sap_lb_cost,1.000000,-0.363952
ytm_gross_profit_amt,-0.363952,1.000000


In [290]:
sap['is_operation'].unique()

array(['非操作型', '操作型'], dtype=object)

In [292]:
corr_opera = sap['is_operation'].replace('操作型', 1).replace('非操作型', 0)
corr_opera = pd.concat([corr_opera, sap['ytm_gross_profit_amt']], axis = 1)
coach_corr = corr_opera.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

,is_operation,ytm_gross_profit_amt
is_operation,1.000000,0.093937
ytm_gross_profit_amt,0.093937,1.000000


In [ ]:

# 不同象限WMS客户，其inbound/outbound量是否有显著区别？
smf.ols(
    'ytm_gross_profit_amt ~ inbound_line_count +\
         inbound_receive_qty + outbound_line_count + outbound_shipped_qty ' # + stock_turn_monthly + location_usage_rate
         , data = sap).fit().summary()

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary()